In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE  # SMOTE para aumento de datos
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Para la matriz de confusión

# Cargar el conjunto de datos
data = pd.read_csv('data.csv')

# Eliminar las columnas 'Breed2' y 'Color3'
data = data.drop(columns=['Breed2', 'Color3'])

# Separar características y etiqueta objetivo
X = data.drop(columns=['AdoptionSpeed'])
y = data['AdoptionSpeed']

# Variables numéricas
numeric_features = ['Age', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
                    'Sterilized', 'Health', 'Fee', 'PhotoAmt']

# Variables categóricas para embeddings
embedding_features = ['Breed1', 'Color1', 'Color2', 'State']

# Variables categóricas para codificación one-hot
onehot_features = ['Type', 'Gender']

# Eliminar columnas que no aportan al modelo
X = X.drop(columns=['Name', 'PetID'])

# Reemplazar valores NaN por '-'
X = X.fillna('-')

# Codificar todas las variables categóricas (Label Encoding para embeddings, OneHot para otras)
# Label Encoding para las variables categóricas que se usarán como embeddings
for feature in embedding_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature].astype(str))  # Codificar todas las columnas de embeddings

# One-Hot Encoding para variables como 'Type' y 'Gender'
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_onehot_encoded = onehot_encoder.fit_transform(X[onehot_features].astype(str))

# Concatenar las variables numéricas con las variables codificadas
X_combined = np.concatenate([X[numeric_features].values, X[embedding_features].values, X_onehot_encoded], axis=1)

# Dividir el DataFrame en conjuntos de entrenamiento y validación
X_train_combined, X_val_combined, y_train, y_val = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Aumento de Datos usando SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_combined, y_train)

# Normalizar las características numéricas después del aumento de datos
scaler = StandardScaler()
X_train_numeric_resampled = scaler.fit_transform(X_train_resampled[:, :len(numeric_features)])
X_val_numeric = scaler.transform(X_val_combined[:, :len(numeric_features)])

# Preparar los datos de entrada para el modelo
X_train_inputs_resampled = X_train_resampled

# Crear las capas del modelo de Keras, utilizando las entradas de SMOTE
numeric_input = Input(shape=(X_train_numeric_resampled.shape[1],), name='numeric_input')

# Construir el modelo con regularización L2 y Dropout
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(numeric_input)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)

# Capa de salida
output = Dense(5, activation='softmax', name='output')(x)

# Definir el modelo
model = Model(inputs=numeric_input, outputs=output)

# Compilar el modelo con categorical_crossentropy y accuracy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convertir y_train_resampled e y_val a categorías (one-hot encoding para la salida)
y_train_cat = tf.keras.utils.to_categorical(y_train_resampled, num_classes=5)
y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes=5)

# Callback para Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Callback para ajustar el learning rate si el val_loss no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Entrenar el modelo
history = model.fit(
    x=X_train_numeric_resampled,
    y=y_train_cat,
    validation_data=(X_val_numeric, y_val_cat),
    epochs=50,
    batch_size=80,
    callbacks=[early_stopping, reduce_lr]
)

# Graficar la pérdida (loss) por época
plt.plot(history.history['loss'], label='Pérdida en entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en validación')
plt.title('Pérdida por época')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (Loss)')
plt.legend()
plt.show()

# Graficar la precisión (accuracy) por época
plt.plot(history.history['accuracy'], label='Precisión en entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en validación')
plt.title('Precisión por época')
plt.xlabel('Épocas')
plt.ylabel('Precisión (Accuracy)')
plt.legend()
plt.show()

# ---- Matriz de confusión ----
# Hacer predicciones sobre el conjunto de validación
y_val_pred = model.predict(X_val_numeric)

# Convertir las predicciones de probabilidades a etiquetas
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(y_val, y_val_pred_classes)

# Visualizar la matriz de confusión
confusion_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
confusion_display.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión")
plt.show()
